In [ ]:
# cadCAD standard dependencies

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

# Included with cadCAD
import pandas as pd

# Additional dependencies

# For analytics
import numpy as np
# For visualization
import plotly.express as px

# For using the balancer pool model
from BalancerV2cad.WeightedPool import WeightedPool
from BalancerV2cad.WeightedMath import WeightedMath

from BalancerV2cad.util import *
from BalancerV2cad.BalancerConstants import *


from decimal import Decimal

In [ ]:
# Additional dependencies

# For parsing the data from the API
import json
# For downloading data from API
import requests as req
# For generating random numbers
import math
# For visualization
import plotly.express as px
# For Google BigQuery authentication
from google.oauth2 import service_account

import random

## Helper Functions

In [ ]:
# Helper function for changing pool weight(s)
def changePoolWeights(wp, new_weights):
    new_pool = WeightedPool()
    balances = wp._balances
    new_pool.join_pool(balances,new_weights)
    new_pool.factory_fees = wp.factory_fees
    return new_pool

In [ ]:
# Assuming adjacent CEXs with 'infinite' depth with `cex_trade_slippage` slippage,
# calculates the profitability of performing an arbitrage between the Balancer pool
# and the equivalent market on the CEX.
def calcArbOp(wp, cex_price, cex_trade_fee, cex_trade_slippage):
    num =wp._balances['b'] / wp._weights['b']
    denom = wp._balances['a'] / wp._weights['a']
    current_pool_price = num/denom
    
    # Buying ETH from the pool. Selling on cex.
    if current_pool_price < cex_price:
        effective_cex_price = (1-cex_trade_fee-cex_trade_slippage)*cex_price
        
        def func(x):
            return [
                    divDown(
                        divDown(
                            wp._balances['b']+mulDown( #Num
                                Decimal(x[0]), 1+wp._swap_fee), #Num
                            wp._weights['b'] #Denom
                        ), 
                        divDown( #Denom
                            wp._balances['a']-WeightedMath.calc_out_given_in(
                                wp._balances['b'],
                                wp._weights['b'],
                                wp._balances['a'],
                                wp._weights['a'],
                                Decimal(x[0]) #Num
                            ),
                            wp._weights['a']))-Decimal(effective_cex_price) #Denom
            ]
        amountIn = Decimal(fsolve(func, [1.0])[0])
        return {
            'assetIn': 'b',
            'assetOut': 'a',
            'amountIn': amountIn
        }
    
    # Buying ETH on cex. Selling to the pool.
    elif current_pool_price > cex_price:
        effective_cex_price = (1+cex_trade_fee+cex_trade_slippage)*cex_price
        
        def func(x):
            return [
                    divDown(
                        divDown(
                            wp._balances['b']-WeightedMath.calc_out_given_in(
                                wp._balances['a'],
                                wp._weights['a'],
                                wp._balances['b'],
                                wp._weights['b'],
                                Decimal(x[0])
                            ), #Num
                            wp._weights['b'] #Denom
                        ), 
                        divDown( #Denom
                            wp._balances['a']+mulDown(
                                Decimal(x[0]), 1+wp._swap_fee), #Num
                            wp._weights['a']))-Decimal(effective_cex_price) #Denom
            ]
        amountIn = Decimal(fsolve(func, [1.0])[0])
        return {
            'assetIn': 'a',
            'assetOut': 'b',
            'amountIn': amountIn
        }

## Setup / Preperation Steps

### Acquire the 1M price history from Binance

In [ ]:
ETHUSDT_1M = pd.read_csv("./data/Binance_ETHUSDT_minute.csv")

## Data Wrangling the Data

In [ ]:
ETHUSDT_1M['Date'] = pd.to_datetime(ETHUSDT_1M['date'], format='%Y-%m-%d %H:%M:%S')
ETHUSDT_1M = ETHUSDT_1M.sort_values(by='date', ascending=True)

ETHUSDT_1M = ETHUSDT_1M.reset_index(drop=True)
ETHUSDT_1M.set_index('Date', inplace=True)
ETHUSDT_1M.drop(['date'], axis = 1, inplace=True)

# Truncate off all data from prior to June 2021.
ts = "2021-06-15 00:00:00"
ETHUSDT_1M = ETHUSDT_1M[ETHUSDT_1M.index.date >= pd.to_datetime(ts, format='%Y-%m-%d %H:%M:%S')]

# Add a timestep column
ETHUSDT_1M['timestep'] = np.arange(ETHUSDT_1M.shape[0])

In [ ]:
print(ETHUSDT_1M.tail(5))

In [ ]:
ETHUSDT_1M[ETHUSDT_1M['timestep'] == 0]

In [ ]:
px.line(ETHUSDT_1M,
        x=ETHUSDT_1M.index,
        y='close')

## State Variables

## 1. State Variables

In [ ]:
initial_price = ETHUSDT_1M.iloc[0]['open']
print(initial_price)

In [ ]:
wp = WeightedPool()
wp._swap_fee = Decimal(0.003)
wp.join_pool({'a':1000,'b':1000*initial_price},{'a':0.5,'b':0.5})

#print(wp.swap('b', 'a', 1, given_in=False))
print(wp._balances['a'], wp._balances['b'], wp.factory_fees)

In [ ]:
initial_state = {
    'weighted_pool': wp,
    'swap_fee': wp._swap_fee,
    'external_price': Decimal(initial_price),
}
initial_state

## 2. System Parameters

In [ ]:
system_params = {
    'weights_update_freq': [-1, 30, 10, 5, 1],
#    'weights_update_freq': [-1, 1],
#     'gas_cost': [Decimal(10)] # USD denom
}

## 3. Policy Functions

In [ ]:
def p_price_update(params, substep, state_history, previous_state):
    """
    Calculate cumulative transaction fees & swaps
    from a swap event
    """
    t = previous_state['timestep']
    
    ts_data = ETHUSDT_1M.iloc[t]
    
    timestamp = ts_data.name
    price = ts_data.open
    
    return {
        'timestamp': timestamp,
        'binance_price': price,
    }

## 4. State Update Functions

In [ ]:
def s_timestamp(params,
                   substep,
                   state_history,
                   previous_state,
                   policy_input):
    value = policy_input['timestamp']
    return ('timestamp', value)

def s_external_price(params,
                   substep,
                   state_history,
                   previous_state,
                   policy_input):
    value = Decimal(policy_input['binance_price'])
    return ('external_price', value)

# def s_standard_gas_price(params,
#                    substep,
#                    state_history,
#                    previous_state,
#                    policy_input):
#     return None

In [ ]:
# Arbitrager re-balancing the pool.
def s_primal_arbitrage_rebalance(params,
                   substep,
                   state_history,
                   previous_state,
                   policy_input):
    num = previous_state['weighted_pool']._balances['b'] / previous_state['weighted_pool']._weights['b']
    denom = previous_state['weighted_pool']._balances['a'] / previous_state['weighted_pool']._weights['a']
    current_pool_price = num/denom
    
    # Sell 1 ETH to the pool
    if (policy_input['binance_price'] < Decimal(0.99) * current_pool_price):
        previous_state['weighted_pool'].swap('a', 'b', 1, given_in=True)
    
    # Buy 1 ETH from the pool
    elif (policy_input['binance_price'] > Decimal(1.01) * current_pool_price):
        previous_state['weighted_pool'].swap('b', 'a', 1, given_in=False)
        
    variable = 'weighted_pool'
    return (variable, previous_state['weighted_pool'])

In [ ]:
# Arbitrager re-balancing the pool.
# This agent will perform a pool swap if the profitablility margin exceeds a min. req.
# Parameterize this profitability margin.
def s_sophisticated_arbitrage_rebalance(params,
                   substep,
                   state_history,
                   previous_state,
                   policy_input):
    trade = calcArbOp(previous_state['weighted_pool'], policy_input['binance_price'], 0.001, 0.001)
    if trade:
        if trade['amountIn'] > 0:
            previous_state['weighted_pool'].swap(trade['assetIn'], trade['assetOut'], trade['amountIn'], given_in=True)
        
    variable = 'weighted_pool'
    return (variable, previous_state['weighted_pool'])
    

In [ ]:
# Pool Operator changing the pool weights.
# Open-loop primal variant
# Naive assumption of zero gas fees

from scipy.optimize import fsolve
import numpy as np

# Stop when difference between weight-adjusted price is within 0.05% of new_price (< 0.0005x)
def find_optimal_weights(a_bal, a_weight, b_bal, b_weight, new_price):
    def func(x):
        return [x[0] + x[1] - 1,
            b_bal*x[0] - new_price*a_bal*x[1]]
    root = fsolve(func, [a_weight, b_weight])
    return root

def s_dynamic_weights_adjustment(params,
                   substep,
                   state_history,
                   previous_state,
                   policy_input):
    
    num = previous_state['weighted_pool']._balances['b'] / previous_state['weighted_pool']._weights['b']
    denom = previous_state['weighted_pool']._balances['a'] / previous_state['weighted_pool']._weights['a']
    current_pool_price = num/denom

    if params['weights_update_freq'] != -1 and (previous_state['timestep']+1) % params['weights_update_freq'] == 0:
    #if params['weights_update_freq'] != -1 and abs((previous_state['external_price']-current_pool_price)/current_pool_price) >= 0.00:

        a_bal = np.float64(previous_state['weighted_pool']._balances['a'])
        b_bal = np.float64(previous_state['weighted_pool']._balances['b'])
        a_weight = np.float64(previous_state['weighted_pool']._weights['a'])
        b_weight = np.float64(previous_state['weighted_pool']._weights['b'])
        new_price = np.float64(previous_state['external_price'])
        
        
#         res = random.randint(0,1)
#         if res: adj=-1*new_price*0.002
#         else: adj=-1*new_price*0.002
        new_a_weight, new_b_weight = find_optimal_weights(a_bal, a_weight, b_bal, b_weight, new_price)

        previous_state['weighted_pool'] = changePoolWeights(previous_state['weighted_pool'], {'a': new_a_weight, 'b': new_b_weight})

    variable = 'weighted_pool'
    return (variable, previous_state['weighted_pool'])

In [ ]:
# Update last after any pool adjustments performed by agent(s)
# def s_pool_price(params,
#                    substep,
#                    state_history,
#                    previous_state,
#                    policy_input):
#     variable = 'a_price'
#     num = previous_state['weighted_pool']._balances['b'] / previous_state['weighted_pool']._weights['b']
#     denom = previous_state['weighted_pool']._balances['a'] / previous_state['weighted_pool']._weights['a']
#     value = Decimal(num/denom)
#     return (variable, value)

## 5. Partial State Update Block

In [ ]:
partial_state_update_blocks = [
    {
        'label': 'Price Update & Arbitrage rebalance',
        'policies': {
            'policy_price_update': p_price_update
        },
        'variables': {
            'timestamp': s_timestamp,
            'external_price': s_external_price,
            'weighted_pool': s_sophisticated_arbitrage_rebalance,
        }
    },
    {
        'label': 'Dynamic Weights Adjustment',
        'policies': {
            
        },
        'variables': {
            'weighted_pool': s_dynamic_weights_adjustment,
        }
    }
]

# Simulation

## 6. Configuration

In [ ]:
sim_config = config_sim({
    'N': 1,
    'T': range(ETHUSDT_1M.shape[0]),
    'M': system_params
})

In [ ]:
del configs[:] # Clear any prior configs

In [ ]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

## 7. Execution

In [ ]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

## 8. Output Preperation

In [ ]:
simulation_result = pd.DataFrame(raw_result)

In [ ]:
df = simulation_result.copy()
df = df[df.simulation == 0]

In [ ]:
get_a_balance = lambda x: x._balances['a']
get_b_balance = lambda x: x._balances['b']
get_a_weight = lambda x: x._weights['a']
get_b_weight = lambda x: x._weights['b']
get_a_ffees = lambda x: x.factory_fees['a']
get_b_ffees = lambda x: x.factory_fees['b']

df['a_balance'] = df['weighted_pool'].apply(get_a_balance)
df['b_balance'] = df['weighted_pool'].apply(get_b_balance)
df['a_weight'] = df['weighted_pool'].apply(get_a_weight)
df['b_weight'] = df['weighted_pool'].apply(get_b_weight)
df['a_price'] = (df['b_balance']/df['b_weight'])/(df['a_balance']/df['a_weight'])
df['a_ffees'] = df['weighted_pool'].apply(get_a_ffees)
df['b_ffees'] = df['weighted_pool'].apply(get_b_ffees)
df['cumsum_fees'] = df['a_ffees'] * df['external_price'] + df['b_ffees']

initial_tvl = initial_state['weighted_pool']._balances['a'] * initial_state['external_price'] + initial_state['weighted_pool']._balances['b']
df['tvl'] = df['a_balance'] * df['external_price'] + df['b_balance']
df['tvl_perc_dif'] = (df['tvl'] - initial_tvl) / initial_tvl

df['initial_pos_value'] = initial_state['weighted_pool']._balances['a'] * df['external_price'] + initial_state['weighted_pool']._balances['b']
df['il'] = ((df['tvl'] - df['cumsum_fees']) - df['initial_pos_value'])
df['il_perc_dif'] = df['il'] / df['initial_pos_value']

# Sanity check: ((df['initial_pos_value'] - df['tvl']) + df['il']) + df['cumsum_fees'] ~= 0

In [ ]:
df[df['subset']==0]

In [ ]:
px.line(df, x='timestamp', y='il', facet_row='subset', title='Imperm Loss')

In [ ]:
px.line(df, x='timestamp', y=['tvl', 'initial_pos_value'], facet_row='subset')

In [ ]:
px.line(df, x='timestamp', y=['a_price', 'external_price'], facet_row='subset')

In [ ]:
px.line(df, x='timestamp', y=['a_weight', 'b_weight'], facet_row='subset')

In [ ]:
px.line(df, x='timestamp', y=['il_perc_dif'],)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df[df['subset']==0].timestamp,
    y=df[df['subset']==0].il_perc_dif,
    name="N/A"
))

fig.add_trace(go.Scatter(
    x=df[df['subset']==0].timestamp,
    y=df[df['subset']==1].il_perc_dif,
    name="Every 30 mins"
))

fig.add_trace(go.Scatter(
    x=df[df['subset']==0].timestamp,
    y=df[df['subset']==2].il_perc_dif,
    name="Every 10 mins"
))

fig.add_trace(go.Scatter(
    x=df[df['subset']==0].timestamp,
    y=df[df['subset']==3].il_perc_dif,
    name="Every 5 mins"
))

fig.add_trace(go.Scatter(
    x=df[df['subset']==0].timestamp,
    y=df[df['subset']==4].il_perc_dif,
    name="Every 1 min"
))


fig.show()

In [ ]:
df[df['subset']==0].timestamp